**Imports**

In [1]:
from main import *

c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
save_config(["C:\\Users\\Asus\\Desktop\\Phosphene.AI\\timesformer\\MINTIME-Multi-Identity-size-iNvariant-TIMEsformer-for-Video-Deepfake-Detection\\examples\\fake_1_face_0.mp4"])

C:\Users\Asus\Desktop\Phosphene.AI\timesformer\MINTIME-Multi-Identity-size-iNvariant-TIMEsformer-for-Video-Deepfake-Detection\examples\fake_1_face_0.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.
The video is fake (100.0%), showing video result...


['./fake_1_face_0.avi']

**CONFIGS & CONSTANTS**

In [4]:
model_path = "pretrained/MINTIME_XC_Model_checkpoint30"
extractor_path = "pretrained/MINTIME_XC_Extractor_checkpoint30"
video_path = "videos/Alice Weidel  - ARD Tagesthemen Deepfake  - Compare  Version.mp4"
RANGE_SIZE = 5
SIZE_EMB_DICT = [(1+i*RANGE_SIZE, (i+1)*RANGE_SIZE) if i != 0 else (0, RANGE_SIZE) for i in range(25)]

**UTILS**

In [5]:
def get_identity_information(identity, faces):
    mean_side = mean([row[1].size[0] for row in faces])   
    number_of_faces = len(faces)
    return [identity, mean_side, number_of_faces, faces]

def get_sorted_identities(identities, discarded_faces, max_identities = 2, num_frames = 16):
    sorted_identities = []
    discarded_faces = []
    for identity in identities:
        sorted_identities.append(get_identity_information(identity, identities[identity]))


    # Sort identities based on faces size
    sorted_identities = sorted(sorted_identities, key=lambda x:x[1], reverse=True)

    if len(sorted_identities) > max_identities:
        sorted_identities = sorted_identities[:max_identities]

    # Adjust the identities list faces number
    identities_number = len(sorted_identities)
    available_additional_faces = []
    if identities_number > 1:
        max_faces_per_identity = {1: [num_frames], 
                  2:  [int(num_frames/2), int(num_frames/2)],
                  3:  [int(num_frames/3), int(num_frames/3), int(num_frames/4)],
                  4:  [int(num_frames/3), int(num_frames/3), int(num_frames/8), int(num_frames/8)]}

        max_faces_per_identity = max_faces_per_identity[identities_number]
        for i in range(identities_number):
            if sorted_identities[i][2] < max_faces_per_identity[i] and i < identities_number - 1:
                sorted_identities[i+1][2] += max_faces_per_identity[i] - sorted_identities[i][2] 
                available_additional_faces.append(0)
            elif sorted_identities[i][2] > max_faces_per_identity[i]:
                available_additional_faces.append(sorted_identities[i][2] - max_faces_per_identity[i])
                sorted_identities[i][2] = max_faces_per_identity[i]
            else:
                available_additional_faces.append(0)

    else: # If only one identity is in the video, all the frames are assigned to this identity
        sorted_identities[0][2] = num_frames
        available_additional_faces.append(0)


    # Check if we found enough faces to fullfill the input sequence, otherwise go back and add some faces from previous identities
    input_sequence_length = sum(faces_number for _, _, faces_number, _ in sorted_identities)
    if input_sequence_length < num_frames:
        for i in range(identities_number):
            needed_faces = num_frames - input_sequence_length
            if available_additional_faces[i] > 0:
                added_faces = min(available_additional_faces[i], needed_faces)
                sorted_identities[i][2] += added_faces
                input_sequence_length += added_faces
                if input_sequence_length == num_frames:
                    break
        # If not enough faces have been found, add some "dummy" images in the last identity
        if input_sequence_length < num_frames:
            needed_faces = num_frames - input_sequence_length
            sorted_identities[-1][2] += needed_faces
            input_sequence_length += needed_faces
    
    return sorted_identities, discarded_faces

def create_val_transform(size, additional_targets):
    return Compose([
        IsotropicResize(max_side=size, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC),
        PadIfNeeded(min_height=size, min_width=size, border_mode=cv2.BORDER_CONSTANT),
        Resize(height=size, width=size)
    ],  additional_targets = additional_targets, is_check_shapes= False
    )

def generate_masks(video_path, identities, discarded_faces, num_frames, image_size, num_patches):
    mask = []
    last_range_end = 0
    sequence = []
    size_embeddings = []
    
    images_frames = []
    for identity_index, identity in enumerate(identities):
        max_faces = identity[2]
        identity_images = identity[3]
        

        # Select uniformly the frames in an alternate way
        if len(identity_images) > max_faces:
            idx = np.round(np.linspace(0, len(identity_images) - 2, max_faces)).astype(int)
            images = []
            for i in idx:
                images.append(identity_images[i])
            identity_images = images
            
        images_frames.extend(identity_image[0] for identity_image in identity_images)
        identity_images = [identity_image[1] for identity_image in identity_images]

        # Generate size embeddings
        capture = cv2.VideoCapture(video_path)
        width  = capture.get(3)  
        height = capture.get(4) 
        video_area = width*height/2
        identity_size_embeddings = []
        
        for image_index, image in enumerate(identity_images):
            # Get face-frame area ratio for size embedding
            face_area = image.size[0] * image.size[1]
            ratio = int(face_area * 100 / video_area)
            side_ranges = list(map(lambda a_: ratio in range(a_[0], a_[1] + 1), SIZE_EMB_DICT))
            identity_size_embeddings.append(np.where(side_ranges)[0][0]+1)
      

        # If the readed faces are less than max_faces we need to add empty images and generate the mask
        if len(identity_images) < max_faces: 
            diff = max_faces - len(identity_size_embeddings)
            identity_size_embeddings = np.concatenate((identity_size_embeddings, np.zeros(diff)))
            identity_images.extend([np.zeros((image_size, image_size, 3), dtype=np.uint8) for i in range(diff)])
            mask.extend([1 if i < max_faces - diff else 0 for i in range(max_faces)])
            images_frames.extend([max(images_frames) for i in range(diff)])
        else: # Otherwise all the faces are valid
            mask.extend([1 for i in range(max_faces)])

        # Compose the size_embedding and sequence list
        size_embeddings.extend(identity_size_embeddings)
        sequence.extend(identity_images)

    # Transform the images, the same transformation is applied to all the faces in the same video
    sequence = [np.asarray(image) for image in sequence]
    additional_targets_keys = ["image" + str(i) for i in range(num_frames)]
    additional_targets_values = ["image" for i in range(num_frames)]
    additional_targets = dict(zip(additional_targets_keys, additional_targets_values))

 
    transform = create_val_transform(image_size, additional_targets)  
    if len(sequence) == 8:
        transformed_images = transform(image=sequence[0], image1=sequence[1], image2=sequence[2], image3=sequence[3], image4=sequence[4], image5=sequence[5], image6=sequence[6], image7=sequence[7])
    elif len(sequence) == 16:
        transformed_images = transform(image=sequence[0], image1=sequence[1], image2=sequence[2], image3=sequence[3], image4=sequence[4], image5=sequence[5], image6=sequence[6], image7=sequence[7], image8=sequence[8], image9=sequence[9], image10=sequence[10], image11=sequence[11], image12=sequence[12], image13=sequence[13], image14=sequence[14], image15=sequence[15])
    else:
        raise Exception("Invalid number of frames.")

    sequence = [transformed_images[key] for key in transformed_images]
        
    # Generate the identities_mask telling to the model which faces attend to an identity and which to another one
    identities_mask = []
    last_range_end = 0
    for identity_index in range(len(identities)):
        identity_mask = [True if i >= last_range_end and i < last_range_end + identities[identity_index][2] else False for i in range(0, num_frames)]
        for k in range(identities[identity_index][2]):
            identities_mask.append(identity_mask)
        last_range_end += identities[identity_index][2]

    # Generate coherent temporal-positional embedding
    images_frames_positions = {k: v+1 for v, k in enumerate(sorted(set(images_frames)))}
    frame_positions = [images_frames_positions[frame] for frame in images_frames]   
    if num_patches != None: 
        positions = [[i+1 for i in range(((frame_position-1)*num_patches), num_patches*(frame_position))] for frame_position in frame_positions]
        positions = sum(positions, []) # Merge the lists
        positions.insert(0,0) # Add CLS
    else:
        positions = []

    tokens_per_identity = [(identities[i][0], identities[i][2]*num_patches + identities[i-1][2]*num_patches) if i > 0 else (identities[i][0], identities[i][2]*num_patches) for i in range(len(identities))]     

    return torch.tensor([sequence]).float(), torch.tensor([size_embeddings]).int(), torch.tensor([mask]).bool(), torch.tensor([identities_mask]).bool(), torch.tensor([positions]), tokens_per_identity


**Detect Faces**

In [6]:
def collate_fn(x):
    return x

def detect_faces(video_path):
    # Init the face detector
    detector = face_detector.FacenetDetector(device="cpu")

    # Read the video and its information
    dataset = VideoDataset([video_path])
    loader = DataLoader(dataset, shuffle=False, num_workers=0, batch_size=1, collate_fn= collate_fn)
    
    # Detect the faces
    for item in loader: 
        bboxes = {}
        video, indices, fps, frames = item[0]
        bboxes.update({i : b for i, b in zip(indices, detector._detect_faces(frames))})
        found_faces = False
        for key in list(bboxes.keys()):
            if type(bboxes[key]) == list:
                found_faces = True
                break

        if not found_faces:
            raise Exception("No faces found.")

    return bboxes

**Crop Faces**

In [7]:
def extract_crops(video_path, bboxes_dict):

    # Read video frames
    frames = []
    
    capture = cv2.VideoCapture(video_path)
    frames_num = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(capture.get(5))

    for i in range(frames_num):
        capture.grab()
        success, frame = capture.retrieve()
        if not success:
            continue
        frames.append(frame)

    # Extract the faces crops
    explored_indexes = []
    crops = []

    for i in range(0, len(frames), fps):
        while str(i) not in bboxes_dict:
            if i == frames_num - 1:
                i -= 1
            if i in explored_indexes:
                break
            else:
                explored_indexes.append(i)

        frame = frames[i]
        index = i
        limit = i + fps - 1
        keys = [int(x) for x in list(bboxes_dict.keys())]

        while index < limit:
            index += 1
            if index in keys and bboxes_dict[index] is not None:
                break
        if index == limit:
            continue

        bboxes = bboxes_dict[index]
        for bbox in bboxes:
            xmin, ymin, xmax, ymax = [int(b * 2) for b in bbox]
            w = xmax - xmin
            h = ymax - ymin

            # Add some padding to catch background too
            p_h = h // 3
            p_w = w // 3
            
            crop_h = (ymax + p_h) - max(ymin - p_h, 0)
            crop_w = (xmax + p_w) - max(xmin - p_w, 0)

            # Make the image square
            if crop_h > crop_w:
                p_h -= int(((crop_h - crop_w)/2))
            else:
                p_w -= int(((crop_w - crop_h)/2))

            # Extract the face from the frame
            crop = frame[max(ymin - p_h, 0):ymax + p_h, max(xmin - p_w, 0):xmax + p_w]
            
            # Check if out of bound and correct
            h, w = crop.shape[:2]
            if h > w:
                diff = int((h - w)/2)
                if diff > 0:         
                    crop = crop[diff:-diff,:]
                else:
                    crop = crop[1:,:]
            elif h < w:
                diff = int((w - h)/2)
                if diff > 0:
                    crop = crop[:,diff:-diff]
                else:
                    crop = crop[:,:-1]

            # Add the extracted face to the list
            crops.append((i, Image.fromarray(crop), bbox))

    return crops

**Clustering function**

In [8]:
def cluster_faces(crops, valid_cluster_size_ratio = 0.20, similarity_threshold = 0.80):

    # Convert crops to PIL images
    crops_images = [row[1] for row in crops]
    
    # Extract the embeddings
    embeddings_extractor = InceptionResnetV1(pretrained='vggface2').eval().to("cuda")
    faces = [preprocess_images(face) for face in crops_images]
    faces = np.stack([np.uint8(face) for face in faces])
    faces = torch.as_tensor(faces)
    faces = faces.permute(0, 3, 1, 2).float()
    faces = fixed_image_standardization(faces)
    face_recognition_input = faces
    embeddings = []
    embeddings = embeddings_extractor(face_recognition_input.to(device)).detach().cpu().numpy()

    # Cluste
    similarities = np.dot(np.array(embeddings), np.array(embeddings).T)
    
    components = _generate_connected_components(
        similarities, similarity_threshold=similarity_threshold
    )
    components = [sorted(component) for component in components]

    clustered_faces = {}
    for identity_index, component in enumerate(components):
        for index, face_index in enumerate(component):
            component[index] = crops[face_index]
        
        clustered_faces[identity_index] = component

    return clustered_faces

**Predict**

In [18]:
def predict(video_path, clustered_faces, config, model_weights,discarded_faces = None, save_attentions = True):

   
    features_extractor = xception(num_classes=1, pretrain_path="pretrained/MINTIME_XC_Extractor_checkpoint30")


    model = SizeInvariantTimeSformer(config=config, require_attention=True)
    
    
    features_extractor = torch.nn.DataParallel(features_extractor)
    model = torch.nn.DataParallel(model)

    features_extractor = features_extractor.to(device)    
    model = model.to(device)
    features_extractor.eval()
    model.eval()

    if os.path.exists(model_weights):
        model.load_state_dict(torch.load(model_weights))
    else:
        raise Exception("No checkpoint loaded for the model.")    
    
    identities, discarded_faces  = get_sorted_identities(clustered_faces, discarded_faces, len(clustered_faces))
    videos, size_embeddings, mask, identities_mask, positions, tokens_per_identity = generate_masks(video_path, identities, discarded_faces, config["model"]["num-frames"], config["model"]["image-size"], config["model"]["num-patches"])
    b, f, h, w, c = videos.shape
    videos = videos.to(device)    
    identities_mask = identities_mask.to(device)
    mask = mask.to(device)
    positions = positions.to(device)
    

    with torch.no_grad():
        video = rearrange(videos, "b f h w c -> (b f) c h w")
        features = features_extractor(video)  

        features = rearrange(features, '(b f) c h w -> b f c h w', b = b, f = f)   
        test_pred, attentions = model(features, mask=mask, size_embedding=size_embeddings, identities_mask=identities_mask, positions=positions)
        
        identity_names = [row[0] for row in tokens_per_identity]
        frames_per_identity = [int(row[1] / config["model"]["num-patches"]) for row in tokens_per_identity]
        
        if save_attentions:
            aggregated_attentions, identity_attentions = aggregate_attentions(attentions, config['model']['heads'], config['model']['num-frames'], frames_per_identity)
            # save_attention_plots(aggregated_attentions, identity_names, frames_per_identity, config['model']['num-frames'], os.path.basename(video_path), out_pth)
        else:
            identity_attentions = []
            aggregated_attentions = []
        return torch.sigmoid(test_pred[0]).item(), identity_attentions, aggregated_attentions, identities, frames_per_identity

**More Utils**

In [11]:
def extract_faces(frame, bboxes):
    xmin, ymin, xmax, ymax = [int(b * 2) for b in bboxes]
    w = xmax - xmin
    h = ymax - ymin

    # Add some padding to catch background too
    p_h = h // 3
    p_w = w // 3
    
    crop_h = (ymax + p_h) - max(ymin - p_h, 0)
    crop_w = (xmax + p_w) - max(xmin - p_w, 0)

    # Make the image square
    if crop_h > crop_w:
        p_h -= int(((crop_h - crop_w)/2))
    else:
        p_w -= int(((crop_w - crop_h)/2))

    # Extract the face from the frame
    crop = frame[max(ymin - p_h, 0):ymax + p_h, max(xmin - p_w, 0):xmax + p_w]
    
    # Check if out of bound and correct
    h, w = crop.shape[:2]
    if h > w:
        diff = int((h - w)/2)
        if diff > 0:         
            crop = crop[diff:-diff,:]
        else:
            crop = crop[1:,:]
    elif h < w:
        diff = int((w - h)/2)
        if diff > 0:
            crop = crop[:,diff:-diff]
        else:
            crop = crop[:,:-1]
    return crop

def get_identities_bboxes(identities):
    identities_bboxes = {}
    for row in identities:
        identity = row[3]
        for face in identity:
            frame = face[0]
            if frame in identities_bboxes:
                identities_bboxes[frame].append(face[2])
            else:
                identities_bboxes[frame] = [face[2]]
    return identities_bboxes


def generate_output_video(video_path, pred, identity_attentions, aggregated_attentions, identities, frames_per_identity, video_dir = ""):

    identities_bboxes = get_identities_bboxes(identities)
    available_frames_keys = [frame for frame in identities_bboxes]
    path = os.path.join("Results1",video_dir,os.path.basename(video_path).replace(".mp4","s"))
    if not os.path.exists(path):
        os.makedirs(path)
    cap = cv2.VideoCapture(video_path)
    width  = cap.get(3)  
    height = cap.get(4) 
    fps = int(cap.get(5))
    fourcc = hex(int(cap.get(cv2.CAP_PROP_FOURCC)))
    output = cv2.VideoWriter("examples/preds/"+str(os.path.basename(video_path).replace(".mp4", ".avi")), cv2.VideoWriter_fourcc("X", "V", "I", "D"), fps, (int(width), int(height)))
    frame_index = 0
    while True:
        config_gg = {}
        ret, frame = cap.read()
        config_gg["frame_number"] = frame_index
        config_gg["identities"] = {}
      
        if ret:
            nearest_frame_index = min(available_frames_keys, key=lambda x:abs(x - frame_index))
            # if nearest_frame_index - frame_index > fps: 
            #     continue
            

            bbox = identities_bboxes[nearest_frame_index]
            for identity_index, identity_bbox in enumerate(bbox):
                identity_path = os.path.join(path,f"frame_{frame_index}",f"identity_{identity_index}")
                if not os.path.exists(identity_path):
                    os.makedirs(identity_path)
                config_gg["identities"][identity_index] = {}
                config_gg["identities"][identity_index]["bbox"] = identity_bbox 
                img = extract_faces(frame, identity_bbox)
                xmin, ymin, xmax, ymax = [int(b * 2) for b in identity_bbox]
                if pred > 0.5:
                    red = 255 * identity_attentions[identity_index]
                    green = 255 - red

                    if red > green:
                        text = 'Fake' + str(round(pred*100,2)) + "%"
                    else:
                        text = 'Pristine'
                    config_gg["identities"][identity_index]["pred_result"] = round(pred*100,2)
                    config_gg["identities"][identity_index]["pred_label"] =  'Fake' if 'Fake' in text else 'Pristine'
                        
                else:
                    green = int(255 * (1 - pred))
                    red = 255 - green
                    text = 'Pristine '  + str(round((1-pred)*100,2)) + "%"
                    config_gg["identities"][identity_index]["pred_result"] = round(pred*100,2)
                    config_gg["identities"][identity_index]["pred_label"] =  'Pristine'




                cv2.imwrite(os.path.join(identity_path,f"{uuid.uuid4()}.jpg"), img)
            
                color = (0, green, red)
                frame = draw_border(frame, (xmin,ymin), (xmax,ymax), color, 2, 10, 20)
                cv2.putText(frame, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                
                # config_gg["identities"][identity_index]["img"] = frame

            config_gg["frame_path"] = os.path.join(path, f"frame_{frame_index}", f"{frame_index}.jpg")
            cv2.imwrite(config_gg["frame_path"], frame)
            with open(os.path.join(path, f"frame_{frame_index}",f"{frame_index}.json"), "w") as f:
                json.dump(config_gg, f, indent = 6)
            output.write(frame)        
        else:
            break
        
        frame_index += 1
    output.release()
    cap.release()

**THIS IS THE MAIN TESTING FUNCTION**

IT SAVES THE VIDEO RESULT TO AN OUTPUT FOLDER NAMED "examples/preds"

This function takes of list of videos as input... to test single videos just give the input as `[video_path]` as in `save_config(["path/to/video.mp4"])`

In [14]:
def save_config(vid):
    count = 0
    for video_path in vid:
        print(video_path)
        bboxes_dict = detect_faces(video_path)
        print("Face detection completed.")
        
        
        print("Cropping faces from the video...")
        crops = extract_crops(video_path, bboxes_dict)
        print("Faces cropping completed.")
        
        
        print("Clustering faces...")
        clustered_faces = cluster_faces(crops)
        print("Faces clustering completed.")

        print("Searching for fakes in the video...")
        pred, identity_attentions, aggregated_attentions, identities, frames_per_identity = predict(video_path, clustered_faces, config, model_path)
        if pred > 0.5:
            print("The video is fake ("+str(round(pred*100,2)) + "%), showing video result...")
        else:
            print("The video is pristine ("+str(round((1-pred)*100,2)) + "%), showing video result...")

        generate_output_video(video_path,pred,identity_attentions, aggregated_attentions, identities, frames_per_identity, f"vid{count}")
        count+=1

**TEST**

In [15]:
video_paths = ["your/video/path/here"]

In [19]:
save_config(video_paths)

VIDEOS/FFIW10K-v1/FFIW10K-v1-release/source/train/train_00000000.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.


/tmp/ipykernel_207281/386711625.py:178: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987290837/work/torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor([sequence]).float(), torch.tensor([size_embeddings]).int(), torch.tensor([mask]).bool(), torch.tensor([identities_mask]).bool(), torch.tensor([positions]), tokens_per_identity


The video is pristine (91.84%), showing video result...
VIDEOS/FFIW10K-v1/FFIW10K-v1-release/source/train/train_00000002.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.
The video is fake (96.16%), showing video result...
VIDEOS/FFIW10K-v1/FFIW10K-v1-release/source/train/train_00000009.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.
The video is fake (94.81%), showing video result...
VIDEOS/FFIW10K-v1/FFIW10K-v1-release/source/train/train_00000012.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.
The video is